# **Signal To Spike Conversion** - Analysis

This notebook conducts an analysis of the Inter-Spike Interval in the UP and DN spike trains to determine the optimal synaptic time constants

### Check WD (change if necessary) and file loading

In [40]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [41]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME =  "seeg_filtered_subset_90-119_segment500_200"    # "filtered_seeg_ch90-119_ch0"    # "seeg_filtered_subset_90-119_segment500_200"
INPUT_FOLDER = f"results/{DATASET_FILENAME}"
RESULTS_FOLDER = f"analysis/{DATASET_FILENAME}"

BASELINE_FILENAME = "seeg_filtered_subset_90-119_segment500_200"

## Load the UP and DN spike trains in the Ripple, FR Band and HFO Band

In [42]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import read_spike_events, MarkerType, band_to_file_name, BaselineAlgorithm

### Load the Baseline Thresholds from the output file from the baseline process

In [43]:
# Specify the chosen Baseline Algorithm
chosen_baseline_alg_suffix = BaselineAlgorithm.SIXTY_FIVE_PERC

In [44]:
# Load the Baseline Thresholds
BASELINE_FILE = f"baseline_results/{BASELINE_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)
baseline_hfo_thresh = round(baseline_thresholds[2], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh
hfo_thresh_up = baseline_hfo_thresh
hfo_thresh_down = -baseline_hfo_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)
print("HFO Thresholds: ", hfo_thresh_up, hfo_thresh_down)

Ripple Thresholds:  4.4523 -4.4523
FR Thresholds:  1.5912 -1.5912
HFO Thresholds:  5.2555 -5.2555


In [45]:
ripple_band_filename = band_to_file_name(MarkerType.RIPPLE)
fr_band_filename = band_to_file_name(MarkerType.FAST_RIPPLE)
hfo_band_filename = band_to_file_name(MarkerType.BOTH)

# Call the function to read the spike events
ripple_up_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_up_spike_train_{ripple_thresh_up}.csv"
ripple_down_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_down_spike_train_{ripple_thresh_down}.csv"

ripple_up_spike_train = read_spike_events(ripple_up_spikes_file_path)
ripple_down_spike_train = read_spike_events(ripple_down_spikes_file_path)


preview_np_array(ripple_up_spike_train, "ripple_up_spike_train", edge_items=3)
preview_np_array(ripple_down_spike_train, "ripple_down_spike_train", edge_items=3)

ripple_up_spike_train Shape: (3669, 2).
Preview: [[ 1.00390625e+03 -1.00000000e+00]
 [ 1.00878906e+03 -1.00000000e+00]
 [ 1.00927734e+03 -1.00000000e+00]
 ...
 [ 1.19043457e+05 -1.00000000e+00]
 [ 1.19044922e+05 -1.00000000e+00]
 [ 1.19055664e+05 -1.00000000e+00]]
ripple_down_spike_train Shape: (3663, 2).
Preview: [[ 1.00585938e+03 -1.00000000e+00]
 [ 1.00683594e+03 -1.00000000e+00]
 [ 1.01220703e+03 -1.00000000e+00]
 ...
 [ 1.19038574e+05 -1.00000000e+00]
 [ 1.19048340e+05 -1.00000000e+00]
 [ 1.19050293e+05 -1.00000000e+00]]


In [46]:
fr_up_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_up_spike_train_{fr_thresh_up}.csv"
fr_down_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_down_spike_train_{fr_thresh_down}.csv"

fr_up_spike_train = read_spike_events(fr_up_spikes_file_path)
fr_down_spike_train = read_spike_events(fr_down_spikes_file_path)

preview_np_array(fr_up_spike_train, "fr_up_spike_train", edge_items=3)
preview_np_array(fr_down_spike_train, "fr_down_spike_train", edge_items=3)

fr_up_spike_train Shape: (8368, 2).
Preview: [[ 1.00048828e+03 -1.00000000e+00]
 [ 1.00244141e+03 -1.00000000e+00]
 [ 1.00488281e+03 -1.00000000e+00]
 ...
 [ 1.19979004e+05 -1.00000000e+00]
 [ 1.19981445e+05 -1.00000000e+00]
 [ 1.19985840e+05 -1.00000000e+00]]
fr_down_spike_train Shape: (8392, 2).
Preview: [[ 1.00097656e+03 -1.00000000e+00]
 [ 1.00341797e+03 -1.00000000e+00]
 [ 1.00390625e+03 -1.00000000e+00]
 ...
 [ 1.19976562e+05 -1.00000000e+00]
 [ 1.19980469e+05 -1.00000000e+00]
 [ 1.19984863e+05 -1.00000000e+00]]


In [47]:
hfo_up_spikes_file_path = f"{INPUT_FOLDER}/{hfo_band_filename}_up_spike_train_{hfo_thresh_up}.csv"
hfo_down_spikes_file_path = f"{INPUT_FOLDER}/{hfo_band_filename}_down_spike_train_{hfo_thresh_down}.csv"

hfo_up_spike_train = read_spike_events(hfo_up_spikes_file_path)
hfo_down_spike_train = read_spike_events(hfo_down_spikes_file_path)

preview_np_array(hfo_up_spike_train, "hfo_up_spike_train", edge_items=3)
preview_np_array(hfo_down_spike_train, "hfo_down_spike_train", edge_items=3)

hfo_up_spike_train Shape: (3064, 2).
Preview: [[ 1.00097656e+03 -1.00000000e+00]
 [ 1.00634766e+03 -1.00000000e+00]
 [ 1.00683594e+03 -1.00000000e+00]
 ...
 [ 1.19036621e+05 -1.00000000e+00]
 [ 1.19050293e+05 -1.00000000e+00]
 [ 1.19487793e+05 -1.00000000e+00]]
hfo_down_spike_train Shape: (3077, 2).
Preview: [[ 1.00244141e+03 -1.00000000e+00]
 [ 1.00292969e+03 -1.00000000e+00]
 [ 1.00488281e+03 -1.00000000e+00]
 ...
 [ 1.19041504e+05 -1.00000000e+00]
 [ 1.19056152e+05 -1.00000000e+00]
 [ 1.19537109e+05 -1.00000000e+00]]


### Load the Annotated Data
The annotated data is needed to separate the intervals of HFO from the normal activity to calculate the Inter-Spike Interval (ISI)

In [48]:
# load the npy object
ripple_ground_truth = np.load(f"{INPUT_FOLDER}/{ripple_band_filename}_ground_truth.npy")
fr_ground_truth = np.load(f"{INPUT_FOLDER}/{fr_band_filename}_ground_truth.npy")
hfo_ground_truth = np.load(f"{INPUT_FOLDER}/{hfo_band_filename}_ground_truth.npy")

preview_np_array(ripple_ground_truth, "ripple_ground_truth", edge_items=3)
preview_np_array(fr_ground_truth, "fr_ground_truth", edge_items=3)
preview_np_array(hfo_ground_truth, "hfo_ground_truth", edge_items=3)

ripple_ground_truth Shape: (197,).
Preview: [('Ripple+Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Ripple', 116037.  , 0.)
 ('Ripple', 116721.  , 0.) ('Spike+Ripple+Fast-Ripple', 119000.  , 0.)]
fr_ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116656.  , 0.)
 ('Spike+Ripple+Fast-Ripple', 119000.  , 0.)]
hfo_ground_truth Shape: (222,).
Preview: [('Ripple+Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Ripple', 116037.  , 0.)
 ('Ripple', 116721.  , 0.) ('Spike+Fast-Ripple', 119000.  , 0.)]


## Iterate the UP and DN Spike Trains to Calculate the Inter-Spike Interval (ISI) in the HFO and Normal Activity

Define the lists to store:
- `Ripple_HFO_UP` - ISI of the UP train in the Ripple band during an HFO event 
- `Ripple_HFO_DN` - ISI of the DN train in the Ripple band during an HFO event
- `Ripple_Noise_UP` - ISI of the UP train in the Ripple band during normal activity
- `Ripple_Noise_DN` - ISI of the DN train in the Ripple band during normal activity

- `FR_HFO_UP` - ISI of the UP train in the Fast Ripple band during an HFO event
- `FR_HFO_DN` - ISI of the DN train in the Fast Ripple band during an HFO event
- `FR_Noise_UP` - ISI of the UP train in the Fast Ripple band during normal activity
- `FR_Noise_DN` - ISI of the DN train in the Fast Ripple band during normal activity

- `BOTH_HFO_UP` - ISI of the UP train in the HFO band during an HFO event
- `BOTH_HFO_DN` - ISI of the DN train in the HFO band during an HFO event
- `BOTH_Noise_UP` - ISI of the UP train in the HFO band during normal activity
- `BOTH_Noise_DN` - ISI of the DN train in the HFO band during normal activity

## Let's analyze the Ripple Band first

In [49]:
isi_ripple_hfo_up = []
isi_ripple_hfo_down = []
isi_ripple_noise_up = []
isi_ripple_noise_down = []

#### Ripple Band - UP Train

In [50]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Iterate the Ripple UP Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True
for spike_idx in range(1, len(ripple_up_spike_train), 1):
    spike_time = ripple_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1

    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > RIPPLE_CONFIDENCE_WINDOW

            print(f"Spike time: {spike_time}. ISI: {isi}. is_first_spike: {is_first_spike}. different_hfo: {different_hfo}")
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_ripple_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_ripple_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

Spike time: 1008.7890625. ISI: 4.8828125. is_first_spike: True. different_hfo: False
Spike time: 1009.27734375. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1009.765625. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1010.25390625. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1015.13671875. ISI: 4.8828125. is_first_spike: False. different_hfo: False
Spike time: 1015.625. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1016.11328125. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1017.08984375. ISI: 0.9765625. is_first_spike: False. different_hfo: False
Spike time: 1022.94921875. ISI: 5.859375. is_first_spike: False. different_hfo: False
Spike time: 1023.4375. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1023.92578125. ISI: 0.48828125. is_first_spike: False. different_hfo: False
Spike time: 1032.2265625. ISI: 8.30078125. is_firs

In [51]:
print(f"Max ISI Ripple UP HFO: {max(isi_ripple_hfo_up)}")
print(f"Min ISI Ripple UP HFO: {min(isi_ripple_hfo_up)}")

Max ISI Ripple UP HFO: 63.96484375
Min ISI Ripple UP HFO: 0.48828125


#### Ripple Band - DOWN Train

In [52]:
# Iterate the Ripple DOWN Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True
for spike_idx in range(1, len(ripple_down_spike_train), 1):
    spike_time = ripple_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > RIPPLE_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_ripple_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_ripple_hfo_down.append(isi)
            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

In [53]:
print(f"Max ISI Ripple DOWN HFO: {max(isi_ripple_hfo_down)}")
print(f"Min ISI Ripple DOWN HFO: {min(isi_ripple_hfo_down)}")

Max ISI Ripple DOWN HFO: 71.2890625
Min ISI Ripple DOWN HFO: 0.48828125


### Validate the number of calculated ISIs
Validate that:
- `len(isi_ripple_hfo_up)` + `len(isi_ripple_noise_up)` = `len(ripple_up)` - 1 (first spike does not have an ISI)
- `len(isi_ripple_hfo_down_)` + `len(isi_ripple_noise_down_)` = `len(ripple_down_)` - 1 (first spike does not have an ISI)

In [54]:
#  Validate that len(isi_ripple_hfo_up) + len(isi_ripple_noise_up) = len(ripple_up)
print(f"Length isi_ripple_hfo_up: {len(isi_ripple_hfo_up)}")
print(f"Length isi_ripple_noise_up: {len(isi_ripple_noise_up)}")
print(f"Sum: {len(isi_ripple_hfo_up) + len(isi_ripple_noise_up)}")
print(f"Length ripple_up: {len(ripple_up_spike_train)}")

print(f"{round(len(isi_ripple_hfo_up)/len(ripple_up_spike_train)*100, 2)}% of the UP spikes are inside the HFO")

Length isi_ripple_hfo_up: 3011
Length isi_ripple_noise_up: 657
Sum: 3668
Length ripple_up: 3669
82.07% of the UP spikes are inside the HFO


In [55]:
#  Validate that len(isi_ripple_hfo_down) + len(isi_ripple_noise_down) = len(ripple_down)
print(f"Length isi_ripple_hfo_down: {len(isi_ripple_hfo_down)}")
print(f"Length isi_ripple_noise_down: {len(isi_ripple_noise_down)}")
print(f"Sum: {len(isi_ripple_hfo_down) + len(isi_ripple_noise_down)}")
print(f"Length ripple_down: {len(ripple_down_spike_train)}")

print(f"{round(len(isi_ripple_hfo_down)/len(ripple_down_spike_train)*100, 2)}% of the DOWN spikes are inside the HFO")

Length isi_ripple_hfo_down: 3010
Length isi_ripple_noise_down: 652
Sum: 3662
Length ripple_down: 3663
82.17% of the DOWN spikes are inside the HFO


## Let's analyze the Fast Ripple Band next

In [56]:
isi_fr_hfo_up = []
isi_fr_hfo_down = []
isi_fr_noise_up = []
isi_fr_noise_down = []

#### Fast Ripple Band - UP Train

In [57]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(fr_up_spike_train), 1):
    spike_time = fr_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > FR_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_fr_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_fr_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

#### Fast Ripple Band - DN Train

In [58]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(fr_down_spike_train), 1):
    spike_time = fr_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            
            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > FR_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_fr_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_fr_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

### Validate the number of calculated ISIs
Validate that:
- `len(isi_fr_hfo_up)` + `len(isi_fr_noise_up)` = `len(fr_up)` - 1 (first spike does not have an ISI)
- `len(isi_fr_hfo_down)` + `len(isi_fr_noise_down)` = `len(fr_down)` - 1 (first spike does not have an ISI)

In [59]:
#  Validate that len(isi_fr_hfo_up) + len(isi_fr_noise_up) = len(ripple_up)
print(f"Length isi_fr_hfo_up: {len(isi_fr_hfo_up)}")
print(f"Length isi_fr_noise_up: {len(isi_fr_noise_up)}")
print(f"Sum: {len(isi_fr_hfo_up) + len(isi_fr_noise_up)}")
print(f"Length fr_up: {len(fr_up_spike_train)}")

print(f"{round(len(isi_fr_hfo_up)/len(fr_up_spike_train)*100, 2)}% of the FR UP spikes are inside the HFO")

Length isi_fr_hfo_up: 3880
Length isi_fr_noise_up: 4487
Sum: 8367
Length fr_up: 8368
46.37% of the FR UP spikes are inside the HFO


In [60]:
#  Validate that len(isi_fr_hfo_down) + len(isi_fr_noise_down) = len(ripple_down)
print(f"Length isi_fr_hfo_down: {len(isi_fr_hfo_down)}")
print(f"Length isi_fr_noise_down: {len(isi_fr_noise_down)}")
print(f"Sum: {len(isi_fr_hfo_down) + len(isi_fr_noise_down)}")
print(f"Length fr_down: {len(fr_down_spike_train)}")

print(f"{round(len(isi_fr_hfo_down)/len(fr_down_spike_train)*100, 2)}% of the FR DOWN spikes are inside the HFO")

Length isi_fr_hfo_down: 3922
Length isi_fr_noise_down: 4469
Sum: 8391
Length fr_down: 8392
46.73% of the FR DOWN spikes are inside the HFO


## Let's analyze the HFO Band next

In [61]:
isi_both_hfo_up = []
isi_both_hfo_down = []
isi_both_noise_up = []
isi_both_noise_down = []

#### HFO Band - UP Train

In [62]:
# Iterate the FR UP Spike Train
curr_hfo_gt_idx = 0  # Current index of the HFO ground truth
prev_spike_time = hfo_up_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(hfo_up_spike_train), 1):
    spike_time = hfo_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the HFO ground truth
    # Skip the ground truth that is before the current spike time
    while curr_hfo_gt_idx < len(hfo_ground_truth) and spike_time > hfo_ground_truth[curr_hfo_gt_idx][1] + BOTH_CONFIDENCE_WINDOW:
        curr_hfo_gt_idx += 1
    
    # Check if the current spike time is within the HFO ground truth
    if curr_hfo_gt_idx < len(hfo_ground_truth):
        curr_gt_start = hfo_ground_truth[curr_hfo_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + BOTH_CONFIDENCE_WINDOW:
            # The spike is within the HFO ground truth

            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > BOTH_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_both_noise_up.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_both_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the HFO ground truth
    isi_both_noise_up.append(isi)
    # Reset the first spike flag
    is_first_spike = True

#### HFO Band - DN Train

In [63]:
# Iterate the FR UP Spike Train
curr_hfo_gt_idx = 0  # Current index of the HFO ground truth
prev_spike_time = hfo_down_spike_train[0][0]

# Track the first spike inside an HFO
is_first_spike = True

for spike_idx in range(1, len(hfo_down_spike_train), 1):
    spike_time = hfo_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the HFO ground truth
    # Skip the ground truth that is before the current spike time
    while curr_hfo_gt_idx < len(hfo_ground_truth) and spike_time > hfo_ground_truth[curr_hfo_gt_idx][1] + BOTH_CONFIDENCE_WINDOW:
        curr_hfo_gt_idx += 1
    
    # Check if the current spike time is within the HFO ground truth
    if curr_hfo_gt_idx < len(hfo_ground_truth):
        curr_gt_start = hfo_ground_truth[curr_hfo_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + BOTH_CONFIDENCE_WINDOW:
            # The spike is within the HFO ground truth
            
            # different_hfo verifies if the current spike is in a different HFO
            different_hfo = isi > BOTH_CONFIDENCE_WINDOW
            if is_first_spike or different_hfo:
                # The first Spike inside an HFO is still considered as noise ISI since the prev_spike_time is outside the HFO
                isi_both_noise_down.append(isi)
                is_first_spike = False
            else:
                # Consider the ISI inside the HFO
                isi_both_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the HFO ground truth
    isi_both_noise_down.append(isi)
    # Reset the first spike flag
    is_first_spike = True

### Validate the number of calculated ISIs
Validate that:
- `len(isi_both_hfo_up)` + `len(isi_both_noise_up)` = `len(hfo_up)` - 1 (first spike does not have an ISI)
- `len(isi_both_hfo_down)` + `len(isi_both_noise_down)` = `len(hfo_down)` - 1 (first spike does not have an ISI)

In [64]:
#  Validate that len(isi_both_hfo_up) + len(isi_both_noise_up) = len(ripple_up)
print(f"Length isi_both_hfo_up: {len(isi_both_hfo_up)}")
print(f"Length isi_both_noise_up: {len(isi_both_noise_up)}")
print(f"Sum: {len(isi_both_hfo_up) + len(isi_both_noise_up)}")
print(f"Length hfo_up: {len(hfo_up_spike_train)}")

print(f"{round(len(isi_both_hfo_up)/len(hfo_up_spike_train)*100, 2)}% of the HFO UP spikes are inside the HFO")

Length isi_both_hfo_up: 2166
Length isi_both_noise_up: 897
Sum: 3063
Length hfo_up: 3064
70.69% of the HFO UP spikes are inside the HFO


In [65]:
#  Validate that len(isi_both_hfo_down) + len(isi_both_noise_down) = len(ripple_down)
print(f"Length isi_both_hfo_down: {len(isi_both_hfo_down)}")
print(f"Length isi_both_noise_down: {len(isi_both_noise_down)}")
print(f"Sum: {len(isi_both_hfo_down) + len(isi_both_noise_down)}")
print(f"Length hfo_down: {len(hfo_down_spike_train)}")

print(f"{round(len(isi_both_hfo_down)/len(hfo_down_spike_train)*100, 2)}% of the HFO DOWN spikes are inside the HFO")

Length isi_both_hfo_down: 2183
Length isi_both_noise_down: 893
Sum: 3076
Length hfo_down: 3077
70.95% of the HFO DOWN spikes are inside the HFO


--- 

## Calculate metrics of the ISIs and Show the Results

In [66]:
# Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_ripple_hfo_up = np.mean(isi_ripple_hfo_up)
median_ripple_hfo_up = np.median(isi_ripple_hfo_up)
std_ripple_hfo_up = np.std(isi_ripple_hfo_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during HFO
q1_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 25)
q3_ripple_hfo_up = np.percentile(isi_ripple_hfo_up, 75)
iqr_ripple_hfo_up = q3_ripple_hfo_up - q1_ripple_hfo_up

# ---- Baseline Activity Metrics ----
mean_ripple_noise_up = np.mean(isi_ripple_noise_up)
median_ripple_noise_up = np.median(isi_ripple_noise_up)
std_ripple_noise_up = np.std(isi_ripple_noise_up)
# Calculate the IQR for the ISI in the Ripple Band for the UP spikes during Noise
q1_ripple_noise_up = np.percentile(isi_ripple_noise_up, 25)
q3_ripple_noise_up = np.percentile(isi_ripple_noise_up, 75)
iqr_ripple_noise_up = q3_ripple_noise_up - q1_ripple_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_ripple_hfo_down = np.mean(isi_ripple_hfo_down)
median_ripple_hfo_down = np.median(isi_ripple_hfo_down)
std_ripple_hfo_down = np.std(isi_ripple_hfo_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during HFO
q1_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 25)
q3_ripple_hfo_down = np.percentile(isi_ripple_hfo_down, 75)
iqr_ripple_hfo_down = q3_ripple_hfo_down - q1_ripple_hfo_down

# ---- Baseline Activity Metrics ----
mean_ripple_noise_down = np.mean(isi_ripple_noise_down)
median_ripple_noise_down = np.median(isi_ripple_noise_down)
std_ripple_noise_down = np.std(isi_ripple_noise_down)
# Calculate the IQR for the ISI in the Ripple Band for the DOWN spikes during Noise
q1_ripple_noise_down = np.percentile(isi_ripple_noise_down, 25)
q3_ripple_noise_down = np.percentile(isi_ripple_noise_down, 75)
iqr_ripple_noise_down = q3_ripple_noise_down - q1_ripple_noise_down

In [67]:
# Fast Ripple band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_fr_hfo_up = np.mean(isi_fr_hfo_up)
median_fr_hfo_up = np.median(isi_fr_hfo_up)
std_fr_hfo_up = np.std(isi_fr_hfo_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during HFO
q1_fr_hfo_up = np.percentile(isi_fr_hfo_up, 25)
q3_fr_hfo_up = np.percentile(isi_fr_hfo_up, 75)
iqr_fr_hfo_up = q3_fr_hfo_up - q1_fr_hfo_up

# ---- Baseline Activity Metrics ----
mean_fr_noise_up = np.mean(isi_fr_noise_up)
median_fr_noise_up = np.median(isi_fr_noise_up)
std_fr_noise_up = np.std(isi_fr_noise_up)
# Calculate the IQR for the ISI in the Fast Ripple Band for the UP spikes during Noise
q1_fr_noise_up = np.percentile(isi_fr_noise_up, 25)
q3_fr_noise_up = np.percentile(isi_fr_noise_up, 75)
iqr_fr_noise_up = q3_fr_noise_up - q1_fr_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_fr_hfo_down = np.mean(isi_fr_hfo_down)
median_fr_hfo_down = np.median(isi_fr_hfo_down)
std_fr_hfo_down = np.std(isi_fr_hfo_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during HFO
q1_fr_hfo_down = np.percentile(isi_fr_hfo_down, 25)
q3_fr_hfo_down = np.percentile(isi_fr_hfo_down, 75)
iqr_fr_hfo_down = q3_fr_hfo_down - q1_fr_hfo_down

# ---- Baseline Activity Metrics ----
mean_fr_noise_down = np.mean(isi_fr_noise_down)
median_fr_noise_down = np.median(isi_fr_noise_down)
std_fr_noise_down = np.std(isi_fr_noise_down)
# Calculate the IQR for the ISI in the Fast Ripple Band for the DOWN spikes during Noise
q1_fr_noise_down = np.percentile(isi_fr_noise_down, 25)
q3_fr_noise_down = np.percentile(isi_fr_noise_down, 75)
iqr_fr_noise_down = q3_fr_noise_down - q1_fr_noise_down

In [68]:
# HFO band ISIs
# UP
# ---- Relevant Event Metrics ----
mean_both_hfo_up = np.mean(isi_both_hfo_up)
median_both_hfo_up = np.median(isi_both_hfo_up)
std_both_hfo_up = np.std(isi_both_hfo_up)
# Calculate the IQR for the ISI in the HFO Band for the UP spikes during HFO
q1_both_hfo_up = np.percentile(isi_both_hfo_up, 25)
q3_both_hfo_up = np.percentile(isi_both_hfo_up, 75)
iqr_both_hfo_up = q3_both_hfo_up - q1_both_hfo_up

# ---- Baseline Activity Metrics ----
mean_both_noise_up = np.mean(isi_both_noise_up)
median_both_noise_up = np.median(isi_both_noise_up)
std_both_noise_up = np.std(isi_both_noise_up)
# Calculate the IQR for the ISI in the HFO Band for the UP spikes during Noise
q1_both_noise_up = np.percentile(isi_both_noise_up, 25)
q3_both_noise_up = np.percentile(isi_both_noise_up, 75)
iqr_both_noise_up = q3_both_noise_up - q1_both_noise_up

# DOWN
# ---- Relevant Event Metrics ----
mean_both_hfo_down = np.mean(isi_both_hfo_down)
median_both_hfo_down = np.median(isi_both_hfo_down)
std_both_hfo_down = np.std(isi_both_hfo_down)
# Calculate the IQR for the ISI in the HFO Band for the DOWN spikes during HFO
q1_both_hfo_down = np.percentile(isi_both_hfo_down, 25)
q3_both_hfo_down = np.percentile(isi_both_hfo_down, 75)
iqr_both_hfo_down = q3_both_hfo_down - q1_both_hfo_down

# ---- Baseline Activity Metrics ----
mean_both_noise_down = np.mean(isi_both_noise_down)
median_both_noise_down = np.median(isi_both_noise_down)
std_both_noise_down = np.std(isi_both_noise_down)
# Calculate the IQR for the ISI in the HFO Band for the DOWN spikes during Noise
q1_both_noise_down = np.percentile(isi_both_noise_down, 25)
q3_both_noise_down = np.percentile(isi_both_noise_down, 75)
iqr_both_noise_down = q3_both_noise_down - q1_both_noise_down

### Print the Metrics of the ISIs

#### Ripple Band

In [69]:
# Metrics of the ISIs in the Ripple Band
print("Ripple UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_up}, Median: {median_ripple_hfo_up}, STD: {std_ripple_hfo_up}, IQR: [{q1_ripple_hfo_up} - {q3_ripple_hfo_up}], Max: {max(isi_ripple_hfo_up)}, Min: {min(isi_ripple_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_up}, Median: {median_ripple_noise_up}, STD: {std_ripple_noise_up}, IQR: [{q1_ripple_noise_up} - {q3_ripple_noise_up}], Max: {max(isi_ripple_noise_up)}, Min: {min(isi_ripple_noise_up)}")

print("=================================")
print("Ripple DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_down}, Median: {median_ripple_hfo_down}, STD: {std_ripple_hfo_down}, IQR: [{q1_ripple_hfo_down} - {q3_ripple_hfo_down}], Max: {max(isi_ripple_hfo_down)}, Min: {min(isi_ripple_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_down}, Median: {median_ripple_noise_down}, STD: {std_ripple_noise_down}, IQR: [{q1_ripple_noise_down}  - {q3_ripple_noise_down}], Max: {max(isi_ripple_noise_down)}, Min: {min(isi_ripple_noise_down)}")

Ripple UP
HFO ISI (ms)
Mean: 3.8992768702258385, Median: 0.9765625, STD: 4.615600919804186, IQR: [0.48828125 - 7.8125], Max: 63.96484375, Min: 0.48828125
Noise ISI (ms)
Mean: 161.81283889840182, Median: 13.18359375, STD: 247.05533695754457, IQR: [8.7890625 - 284.66796875], Max: 2197.265625, Min: 0.48828125
Ripple DOWN
HFO ISI (ms)
Mean: 3.969515676910299, Median: 1.46484375, STD: 4.843162935526565, IQR: [0.48828125 - 7.8125], Max: 71.2890625, Min: 0.48828125
Noise ISI (ms)
Mean: 162.7242199482362, Median: 13.18359375, STD: 249.0977032630668, IQR: [8.7890625  - 284.66796875], Max: 2196.77734375, Min: 0.48828125


#### Fast Ripple Band

In [70]:
# Metrics of the ISIs in the FR Band
print("FR UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_up}, Median: {median_fr_hfo_up}, STD: {std_fr_hfo_up}, IQR: [{q1_fr_hfo_up} - {q3_fr_hfo_up}], Max: {max(isi_fr_hfo_up)}, Min: {min(isi_fr_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_up}, Median: {median_fr_noise_up}, STD: {std_fr_noise_up}, IQR: [{q1_fr_noise_up} - {q3_fr_noise_up}], Max: {max(isi_fr_noise_up)}, Min: {min(isi_fr_noise_up)}")

print("=================================")
print("FR DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_down}, Median: {median_fr_hfo_down}, STD: {std_fr_hfo_down}, IQR: [{q1_fr_hfo_down} - {q3_fr_hfo_down}], Max: {max(isi_fr_hfo_down)}, Min: {min(isi_fr_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_down}, Median: {median_fr_noise_down}, STD: {std_fr_noise_down}, IQR: [{q1_fr_noise_down} - {q3_fr_noise_down}], Max: {max(isi_fr_noise_down)}, Min: {min(isi_fr_noise_down)}")

FR UP
HFO ISI (ms)
Mean: 1.8586148920747423, Median: 0.48828125, STD: 2.2896468803205132, IQR: [0.48828125 - 2.9296875], Max: 40.52734375, Min: 0.48828125
Noise ISI (ms)
Mean: 24.91061417010252, Median: 4.8828125, STD: 61.595048060060144, IQR: [2.44140625 - 21.97265625], Max: 1080.078125, Min: 0.48828125
FR DOWN
HFO ISI (ms)
Mean: 1.883281608554309, Median: 0.48828125, STD: 2.3270633031693717, IQR: [0.48828125 - 2.44140625], Max: 40.0390625, Min: 0.48828125
Noise ISI (ms)
Mean: 24.9715050906243, Median: 4.8828125, STD: 61.738174713111626, IQR: [2.44140625 - 22.4609375], Max: 1082.03125, Min: 0.48828125


#### HFO Band

In [71]:
# Metrics of the ISIs in the FR Band
print("BOTH UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_both_hfo_up}, Median: {median_both_hfo_up}, STD: {std_both_hfo_up}, IQR: [{q1_both_hfo_up} - {q3_both_hfo_up}], Max: {max(isi_both_hfo_up)}, Min: {min(isi_both_hfo_up)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_both_noise_up}, Median: {median_both_noise_up}, STD: {std_both_noise_up}, IQR: [{q1_both_noise_up} - {q3_both_noise_up}], Max: {max(isi_both_noise_up)}, Min: {min(isi_both_noise_up)}")

print("=================================")
print("BOTH DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_both_hfo_down}, Median: {median_both_hfo_down}, STD: {std_both_hfo_down}, IQR: [{q1_both_hfo_down} - {q3_both_hfo_down}], Max: {max(isi_both_hfo_down)}, Min: {min(isi_both_hfo_down)}")
print("Noise ISI (ms)")
print(f"Mean: {mean_both_noise_down}, Median: {median_both_noise_down}, STD: {std_both_noise_down}, IQR: [{q1_both_noise_down} - {q3_both_noise_down}], Max: {max(isi_both_noise_down)}, Min: {min(isi_both_noise_down)}")

BOTH UP
HFO ISI (ms)
Mean: 4.442773257156048, Median: 2.44140625, STD: 5.459300113760427, IQR: [0.9765625 - 7.32421875], Max: 69.3359375, Min: 0.48828125
Noise ISI (ms)
Mean: 121.36429156215162, Median: 7.8125, STD: 211.00725187756314, IQR: [2.44140625 - 194.3359375], Max: 2141.11328125, Min: 0.48828125
BOTH DOWN
HFO ISI (ms)
Mean: 4.308863948694457, Median: 2.44140625, STD: 4.848174486234164, IQR: [0.9765625 - 7.32421875], Max: 68.84765625, Min: 0.48828125
Noise ISI (ms)
Mean: 122.20427544092945, Median: 7.8125, STD: 212.40839778479705, IQR: [2.44140625 - 195.80078125], Max: 2136.71875, Min: 0.48828125


### Find number of outliers in the ISIs

In [72]:
# Print the number of outliers in the ISIs
# Ripple UP
ripple_upper_outliers = len([curr_isi for curr_isi in isi_ripple_hfo_up if curr_isi > q3_ripple_hfo_up + 1.5 * iqr_ripple_hfo_up])

print("Ripple UP HFO Outliers")
print(f"Upper Outliers: {ripple_upper_outliers}/{len(isi_ripple_hfo_up)} ({round(ripple_upper_outliers/len(isi_ripple_hfo_up) * 100, 2)}%)")

# FR UP
fr_upper_outliers = len([curr_isi for curr_isi in isi_fr_hfo_up if curr_isi > q3_fr_hfo_up + 1.5 * iqr_fr_hfo_up])

print("FR UP HFO Outliers")
print(f"Upper Outliers: {fr_upper_outliers}/{len(isi_fr_hfo_up)} ({round(fr_upper_outliers/len(isi_fr_hfo_up) * 100, 2)}%)")

# HFO UP
both_upper_outliers = len([curr_isi for curr_isi in isi_both_hfo_up if curr_isi > q3_both_hfo_up + 1.5 * iqr_both_hfo_up])

print("BOTH UP HFO Outliers")
print(f"Upper Outliers: {both_upper_outliers}/{len(isi_both_hfo_up)} ({round(both_upper_outliers/len(isi_both_hfo_up) * 100, 2)}%)")

Ripple UP HFO Outliers
Upper Outliers: 14/3011 (0.46%)
FR UP HFO Outliers
Upper Outliers: 91/3880 (2.35%)
BOTH UP HFO Outliers
Upper Outliers: 27/2166 (1.25%)


### Display Box Plots with the Inter-Spike Intervals

#### Ripple ISIs

In [73]:
# Display Box Plots with the Inter-Spike Intervals
from utils.bar_plot import create_box_plot
import bokeh.plotting as bplt

# Ripple Band Box Plot
# Define the array of values
ripple_isi_list = [isi_ripple_hfo_up, isi_ripple_hfo_down, isi_ripple_noise_up, isi_ripple_noise_down]
# Create the BoxPlot
ripple_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band", 
    box_arrays=ripple_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_ripple_isi_barplot = True
if show_ripple_isi_barplot:
    # Show the plot
    bplt.show(ripple_isi_boxplot)

ripple_quantiles:  [0.48828125 0.9765625  7.8125    ]
Ripple IQR:  7.32421875
ripple_quantiles:  [0.48828125 1.46484375 7.8125    ]
Ripple IQR:  7.32421875
ripple_quantiles:  [  8.7890625   13.18359375 284.66796875]
Ripple IQR:  275.87890625
ripple_quantiles:  [  8.7890625   13.18359375 284.66796875]
Ripple IQR:  275.87890625


In [74]:
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

# Export the plot to an HTML file
EXPORT_RIPPLE_BOXPLOT = True
if EXPORT_RIPPLE_BOXPLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    ripple_boxplot_filename = f"{RESULTS_FOLDER}/{RIPPLE_BAND_FILENAME}_isi_boxplot_thresh{ripple_thresh_up}-{ripple_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=ripple_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Ripple Band")

    # Save the plot
    bplt.save(ripple_isi_boxplot)

    # Close the plot
    bplt.reset_output()

#### Fast Ripple ISIs

In [75]:
# Display Box Plots with the Inter-Spike Intervals

# FR Band Box Plot
# Define the array of values
fr_isi_list = [isi_fr_hfo_up, isi_fr_hfo_down, isi_fr_noise_up, isi_fr_noise_down]
# Create the BoxPlot
fr_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band", 
    box_arrays=fr_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_fr_isi_barplot = True
if show_fr_isi_barplot:
    # Show the plot
    bplt.show(fr_isi_boxplot)

ripple_quantiles:  [0.48828125 0.48828125 2.9296875 ]
Ripple IQR:  2.44140625
ripple_quantiles:  [0.48828125 0.48828125 2.44140625]
Ripple IQR:  1.953125
ripple_quantiles:  [ 2.44140625  4.8828125  21.97265625]
Ripple IQR:  19.53125
ripple_quantiles:  [ 2.44140625  4.8828125  22.4609375 ]
Ripple IQR:  20.01953125


In [76]:
# Export the plot to an HTML file
EXPORT_FR_PLOT = True
if EXPORT_FR_PLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    fr_boxplot_filename = f"{RESULTS_FOLDER}/{FR_BAND_FILENAME}_isi_boxplot_thresh{fr_thresh_up}-{fr_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=fr_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the Fast Ripple Band")

    # Save the plot
    bplt.save(fr_isi_boxplot)

    bplt.reset_output()

#### HFO ISIs

In [77]:
# Display Box Plots with the Inter-Spike Intervals

# HFO Band Box Plot
# Define the array of values
hfo_isi_list = [isi_both_hfo_up, isi_both_hfo_down, isi_both_noise_up, isi_both_noise_down]
# Create the BoxPlot
hfo_isi_boxplot = create_box_plot(
    title="Inter-Spike Interval of the UP/DOWN Spike trains in the HFO Band", 
    box_arrays=hfo_isi_list,
    y_axis_label='Inter-Spike Interval (ms)',
    x_axis_labels=['HFO UP', 'HFO DOWN', 'NOISE UP', 'NOISE DOWN'],
    sizing_mode="stretch_width",
)
# TODO: Add legend on the x-axis

show_hfo_isi_barplot = True
if show_hfo_isi_barplot:
    # Show the plot
    bplt.show(hfo_isi_boxplot)

ripple_quantiles:  [0.9765625  2.44140625 7.32421875]
Ripple IQR:  6.34765625
ripple_quantiles:  [0.9765625  2.44140625 7.32421875]
Ripple IQR:  6.34765625
ripple_quantiles:  [  2.44140625   7.8125     194.3359375 ]
Ripple IQR:  191.89453125
ripple_quantiles:  [  2.44140625   7.8125     195.80078125]
Ripple IQR:  193.359375


In [78]:
# Export the plot to an HTML file
EXPORT_HFO_PLOT = True
if EXPORT_HFO_PLOT:
    # Create the folder if it does not exist
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)

    hfo_boxplot_filename = f"{RESULTS_FOLDER}/{BOTH_BAND_FILENAME}_isi_boxplot_thresh{hfo_thresh_up}-{hfo_thresh_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=hfo_boxplot_filename, title="Inter-Spike Interval of the UP/DOWN Spike trains in the HFO Band")

    # Save the plot
    bplt.save(hfo_isi_boxplot)

    bplt.reset_output()